In [1]:
from dotenv import load_dotenv
from pathlib import Path
import requests
import os
import re

In [2]:
load_dotenv(dotenv_path="/Users/perso/Documents/Agents/Agentic-Systems/.venv/.env")
guardian_key = os.environ.get("GUARDIAN_APIKEY")

In [3]:
def download_guardian_articles(nb_articles, sujet):

    ENV_PATH = "/Users/perso/Documents/Agents/Agentic-Systems/.venv/.env"
    DATA_DIR = "/Users/perso/Documents/Agents/Agentic-Systems/data/raw"
    os.makedirs(DATA_DIR, exist_ok=True)

    url = "https://content.guardianapis.com/search"

    params = {
        "q": sujet,
        "page-size": nb_articles,
        "api-key": guardian_key,
        "show-fields": "headline,body,byline"
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    results = response.json().get("response", {}).get("results", [])

    def slugify(text):
        text = text.strip().lower()
        text = re.sub(r'[^a-z0-9_\-]+', '-', text)
        text = re.sub(r'-{2,}', '-', text)
        return text.strip('-')

    for article in results:
        fields = article.get('fields', {})
        titre = fields.get('headline', 'Sans titre')
        body = fields.get('body', '')
        date = article.get('webPublicationDate', 'Date inconnue')
        author = fields.get('byline', 'Auteur inconnu')
        journal = article.get('sectionName', 'The Guardian')

        meta_info = (
            f"<div>"
            f"<strong>Date :</strong> {date} &nbsp; | &nbsp; "
            f"<strong>Auteur :</strong> {author} &nbsp; | &nbsp; "
            f"<strong>Journal :</strong> {journal}</div>\n"
        )

        html_content = f"<h1>{titre}</h1>\n{meta_info}{body}"

        filename = f"{slugify(titre)}.html"
        filepath = Path(DATA_DIR) / filename

        if filepath.exists():
            print(f"[SKIP] Article déjà présent : {filename}")
            continue

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(html_content)

In [83]:
download_guardian_articles(nb_articles=100, sujet="")